In [13]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats

In [4]:
SUPPLEMENTARY_TABLES_PATH="/Users/ochapman/projects/pedpancan_ecdna/data/Supplementary Tables.xlsx"

def import_patients():
    return pd.read_excel(SUPPLEMENTARY_TABLES_PATH,sheet_name="1. Patients",index_col=0)
def import_biosamples():
    return pd.read_excel(SUPPLEMENTARY_TABLES_PATH,sheet_name="2. Biosamples",index_col=0)
def import_amplicons():
    return pd.read_excel(SUPPLEMENTARY_TABLES_PATH,sheet_name="3. Amplicons")
def import_genes():
    return pd.read_excel(SUPPLEMENTARY_TABLES_PATH,sheet_name="4. Gene amplifications")
BIOSAMPLES = import_biosamples()

In [54]:
def import_cbtn_TP53():
    path="data/pbta_TP53_alterations.xlsx"
    df = pd.read_excel(path)
    df = df.replace("not profiled",np.nan)
    df = df.dropna(subset=df.columns[4:], how='all')
    # aggregate to patient level
    gb = df.groupby('Patient ID')['Altered'].sum().reset_index()
    gb['TP53_altered'] = gb.Altered > 0
    gb = gb.set_index("Patient ID")
    return gb['TP53_altered']

def import_cbtn_TP53_pathway():
    path="data/alterations_across_samples.tsv"
    df = pd.read_csv(path,sep='\t')
    df = df.replace("not profiled",np.nan)
    df = df.dropna(subset=df.columns[4:], how='all')
    gb = df.groupby('Patient ID')['Altered'].sum().reset_index()
    gb['TP53_altered'] = gb.Altered > 0
    gb = gb.set_index("Patient ID")
    return gb['TP53_altered']

def import_hgg_bs():
    hgg_bs = import_biosamples()
    hgg_bs = hgg_bs[(hgg_bs.cancer_type == "HGG") & (hgg_bs.in_unique_patient_set)]
    return hgg_bs
def import_hgg_pt():
    hgg_pt = import_patients()
    #hgg_pt = hgg_pt[(hgg_pt.cancer_type == "HGG")]
    hgg_pt = hgg_pt.merge(import_cbtn_TP53_pathway(),how='inner',left_index=True,right_index=True)
    return hgg_pt



In [28]:
# Association between TP53 and ecDNA?
hgg_bs = import_hgg_bs()
tmp = hgg_bs[~hgg_bs.cohort.str.startswith("SJC")]
contingency_tbl = pd.crosstab(tmp.amplicon_class == "ecDNA",tmp.cancer_subclass.str.contains("TP53"))
print(
scipy.stats.chi2_contingency(contingency_tbl)
)
contingency_tbl
# Wow, so close. are my data correct, and can I get more?

Chi2ContingencyResult(statistic=3.7563815898541106, pvalue=0.05260630402549279, dof=1, expected_freq=array([[36.22641509, 43.77358491],
       [11.77358491, 14.22641509]]))


cancer_subclass,False,True
amplicon_class,,
False,41,39
True,7,19


In [55]:
pt = import_hgg_pt()
contingency_tbl = pd.crosstab(pt.amplicon_class == "ecDNA",pt.TP53_altered)
print(
scipy.stats.chi2_contingency(contingency_tbl)
)
contingency_tbl

Chi2ContingencyResult(statistic=51.20591512864554, pvalue=8.316764453609536e-13, dof=1, expected_freq=array([[649.23170732, 116.76829268],
       [ 45.76829268,   8.23170732]]))


TP53_altered,False,True
amplicon_class,,
False,668,98
True,27,27


In [22]:
sum(~hgg_bs.patient_id.duplicated())

158

In [56]:
path="data/alterations_across_samples.tsv"
df = pd.read_csv(path,sep='\t')
len(df)

4095